In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings(action='ignore')

from pandas_datareader import data as pdr
import FinanceDataReader as fdr
import yfinance as yf
from pykrx import stock

# 함수

In [2]:
def myplot(df, nrows=5,  kind='line', title='제목', labels=False):  
#     if nrows > 0:
#         print(df.head(nrows))
#     else:
#         print(df.tail(nrows))
    if labels:
        cols = df.columns
        for i, col in enumerate(cols):
            df[col].plot(label=labels[i], kind=kind)
    else :
        df.plot(kind=kind)
    
    plt.title(title)
    plt.legend()
    plt.show()


In [3]:
def myplotmix(df1, df2, y1='ylabel-1', y2='ylabel-2', kind1='bar', kind2='line', title='제목',  nrows1=5, nrows2=5,  labels=False):  
    ytl=['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

    if nrows1 > 0:
        print(df1.head(nrows1))
    elif nrows1 < 0:
        print(df1.tail(nrows1))
    if nrows2 > 0:
        print(df2.head(nrows2))
    else:
        print(df2.tail(nrows2))
    fig, ax1 = plt.subplots(figsize=(12, 4))
    ax2=ax1.twinx()
    if kind1 == 'line':
        df1.plot(kind=kind1, ax=ax2, marker='d',color='#e35f62')  #color='y', 
        df2.plot(kind=kind2, ax=ax1)  #bar
    else:
        df1.plot(kind=kind1, ax=ax1,color='#bcbd22')
        df2.plot(kind=kind2, ax=ax2) #color='y',
        
    # df['date'] = df['date'].dt.strftime('%Y-%m-%d') 
    # ax1.set_xticklabels(df['date'])    
    ax1.yaxis.tick_right()
    ax2.yaxis.tick_left()
    ax1.set_ylabel(y1) #, color='red')
    ax2.set_ylabel(y2) #, color='red')
    ax1.legend(loc=1)
    ax2.legend(loc=2)
    plt.title(title)
    ax1.grid(False)
    
    plt.show()

In [4]:
# def codefind(name):
#     krx = fdr.StockListing('KRX')
#     search = list(krx['Name'])
#     for i in range(len(krx)):
#         if (search[i]==name):
#             print(krx['Symbol'][i])
#             return

- 메소드변경 
codefind -> my_ticker, my_krx

In [5]:
df_code= pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]
df_code = df_code[['회사명', '종목코드']]
df_code.종목코드 = df_code.종목코드.map('{:06d}'.format)
df_code.head()

names=['삼성전자','LG전자']
code_list = []
for name in names:
    code = df_code.query("회사명=='{}'".format(name))['종목코드'].to_string(index=False)
    code_list.append(code)
print(code_list)

['005930', '066570']


In [6]:
def ticker(names):    
    df_code = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]
    df_code.종목코드 = df_code.종목코드.map('{:06d}'.format)
    df_code = df_code[['회사명', '종목코드']]
    code_list = []
    for name in names:
        code = df_code.query("회사명=='{}'".format(name))['종목코드'].to_string(index=False)
        code_list.append(code)
    return code_list


In [18]:
def krxdata(start, end, *codes):
    error_list = []
    failed = 0
    success = 0
#     df= pd.DataFrame()
    for code in codes:
        try:
            df = stock.get_market_ohlcv_by_date(start, end, code)
            df.reset_index(inplace=True)
            df.insert(loc=1, column='종목코드', value=code)
            df.columns = ['date', 'code', 'open', 'high', 'low', 'close','volume']
            success += 1
        except ElementTree.ParseError as e:
            error_list.append(e)
            drop_rows(database, table, code)  # 특정 종목코드에서 에러 발생 시 해당 종목코드에 대한 데이터 삭제 후 다음 종목코드로 넘어감
            failed += 1
            continue
        except Exception as e:
            print('Error occurred : ', e)
            failed += 1
            continue
#     print(f'변환갯수:{len(codes)},success:{success}, failed:{failed}')
    return df


# 데이터 수집 
 ## - stock

In [19]:
# 코드 확인
stocks = ['삼성전자','SK하이닉스','현대자동차','NAVER'] # 종목명 오류일 시, 'Series([], '
stock_codes = ticker(stocks)
stock_codes # ['005930', '000660', '005380', '035420']
df = pd.DataFrame()
for i, sname in enumerate(stocks):
    print(sname, stock_codes[i])
    df[sname] = krxdata('20180101','20191231', stock_codes[i])['close']


삼성전자 005930
SK하이닉스 000660
현대자동차 005380
NAVER 035420


In [20]:
df.head()

,삼성전자,SK하이닉스,현대자동차,NAVER
0,51020,76600,149500,177251
1,51620,77700,150500,174447
2,51080,77100,146500,178853
3,52120,79300,149000,181857
4,52020,78200,151000,190269


# 수익률
- shift(1) rolling() pch_change()

# 리스크
- 변동성 = 분산(기대(평균)수익률로부터 +- 변동성)
- 변동성 크다 = 리스크 크다
- 공분산

In [ ]:
# 연단위 편차
day_std = dflog[['삼성전자','SK하이닉스']].std()
year_std = day_std * np.sqrt(252)
print(year_std)

In [ ]:
# 연단위 공분산
day_cov = dflog[['삼성전자','SK하이닉스']].cov()
year_cov = day_cov * 252
print(year_cov)

# 투자의 가장 효율적 배분은?

In [ ]:
ws = np.arange(.1, 1.0, .1)

for i in ws:
    w = np.array([i, 1-i])
    res = ym.dot(w)
    print(w, res)

# 효율적 투자선

In [ ]:
dcov= dflog.cov()
ycov= dflog.cov() * 252
dmean= dflog.mean()
ymean= dflog.mean()